# Парсер с Lenta.ru новости


Данная программа парсит новости с https://lenta.ru/search за определенный период

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

md = '/content/drive/My Drive/Colab Notebooks/Diplom/'

In [ ]:
import requests
import threading
from bs4 import BeautifulSoup
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import time
import random

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
days_in_month_dict ={
    1:31,
    2:29,
    3:31,
    4:30,
    5:31,
    6:30,
    7:31,
    8:31,
    9:30,
    10:31,
    11:30,
    12:31
}

number_of_month_dict ={
    1:'01',
    2:'02',
    3:'03',
    4:'04',
    5:'05',
    6:'06',
    7:'07',
    8:'08',
    9:'09',
    10:'10',
    11:'11',
    12:'12'
}

In [ ]:
def soup_item_to_dict(s_item):
  times = s_item.find('span' ,class_="g-date item__date").text
  item_dict = {}
  item_dict['time'] = times[:5]
  if times[8:]==' ':
    item_dict['date'] = times[9:]
  else:
    item_dict['date'] = times[8:]
  item_dict['link'] = 'https://lenta.ru' + s_item.find('a').get('href')
  item_dict['headline'] = s_item.find('div', class_="titles").find('span').text.replace('\xa0', ' ')
  return item_dict

# soup_item_to_dict(topic_news[0])

In [ ]:
lock = threading.Lock()

def take_text_and_tag_news(id, now_link):
  time_delay = time.time()

  tag =''
  st=''
  flag = True
  iterator_0 = 0
  while flag and iterator_0<10:
    iterator_0+=1

    r = requests.get(now_link)
    soup = BeautifulSoup(r.text, 'html.parser')

    try:
      tag = soup.find('a', class_ = 'b-header__block').text
      sp = soup.find('div', class_ = 'b-text')
      st = ''
      for s_ in sp.find_all('p'):
        st+=s_.text + ' '
    except:
      pass

    if tag!='':
      flag = False
    else:
      time.sleep(0.1)

  if tag=='':
    print('tag NOT defined (((')
  time_delay = time.time() - time_delay
  global max_time_delay
  global news
  with lock:
      max_time_delay = max(max_time_delay, time_delay)
      news.extend([(id,tag, st)])

  if id%500==0:
    print(f'{id} {tag}')
  return tag, st

# take_text_and_tag_news(0, 'https://lenta.ru/news/2020/12/01/nervy/')

# Сбор новостей

In [ ]:
# range(1, 13) for full year
for mount_id in tqdm(range(1, 13)):

  topic_news = []
  month = mount_id

  base_url = 'https://lenta.ru/news/2020/0' + str(month) + '/'
  days_in_month = days_in_month_dict[month]
  print(f'{month} - {days_in_month}')
  for day_id in tqdm(range(1, days_in_month+1)):
    now_day_str = str(day_id)
    if len(now_day_str)==1:
      now_day_str='0'+now_day_str
    now_day_str = base_url+now_day_str
    try:
      r = requests.get(now_day_str)
      soup = BeautifulSoup(r.text, 'html.parser')
      now_day_items = soup.find_all('div', class_ =  'item news b-tabloid__topic_news')
      topic_news.extend(now_day_items)

      if len(now_day_items)==0:
        print('now_day_items is NOT define')


    except:
      print(str(day_id)+' error')

  len(topic_news)
  topic_news_dict = [soup_item_to_dict(tn) for tn in topic_news]
  print(f'total len: {len(topic_news_dict)}')


  news = []
  max_time_delay=0
  max_thread_number = 50
  threads_index = set([])


  arr_news_of_time = time.time()

  thrs_main = [threading.Thread(target=take_text_and_tag_news, args= (i, tnd['link'],))
                for i, tnd in enumerate(topic_news_dict)]

  now_threads = set([])

  iterator = 0

  while iterator<len(thrs_main):
    finished_threads=set([])
    for th in now_threads:
      if not th.isAlive():
        finished_threads.add(th)
    now_threads-=finished_threads

    while len(now_threads)<max_thread_number:
      try:
        now_threads.add(thrs_main[iterator])
        thrs_main[iterator].start()
        iterator+=1
        time.sleep(0.01)
      except:
        print(f'iterator: {iterator} total len: {len(thrs_main)}')
        break
    time.sleep(0)


  for ths in thrs_main:
    ths.join()

  print(time.time()-arr_news_of_time)
  print(f'max_time_delay {max_time_delay}')
  len(news)

  for n in news:
    topic_news_dict[n[0]]['tag'] = n[1]
    topic_news_dict[n[0]]['text'] = n[2]


  data = pd.DataFrame(topic_news_dict)
  data.head()


  data.to_pickle(md+"data/news_lenta_"+ number_of_month_dict[mount_id]+ "_2020")

1 - 31


3 error
4 error
5 error
